In [48]:
import numpy as np
import pandas as pd
import altair as alt
from sklearn.decomposition import PCA
from scipy import linalg
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
alt.data_transformers.disable_max_rows()

city_sust = pd.read_csv('data/uscity-sustainability-indices.csv')

# Data Science Concepts and Analysis

### Week 5: Principal components

#### (Exploratory data analysis II)

* Covariation between variables
* Eigendecomposition
* Principal components analysis

## This week: principal components

**Objective**: introduce an exploratory technique for exploring covariation among several variables.

* **Covariation between variables**
    + What is covariation and how is it measured quantitatively?
    + Covariance and correlation
    + Correlation matrix
    + Heatmap visualization

* **Eigendecomposition**
    + The eigenvalue problem
    + Geometric interpretation
    + Computations

* **Principal components analysis**
    + What is PCA exactly?
    + PCA in the low-dimensional setting
    + Variation capture and loss
    + Interpreting principal components
    + Example application

# Covariation between variables

* What is covariation and how is it measured quantitatively?
* Covariance and correlation matrices
* Visualizations

## City sustainability data

Throughout this week's lecture we'll use the following dataset on the sustainability of U.S. cities:

In [2]:
city_sust.head(4)

,GEOID_MSA,Name,Econ_Domain,Social_Domain,Env_Domain,Sustain_Index
0,310M300US10100,"Aberdeen, SD Micro Area",0.565264,0.591259,0.444472,1.600995
1,310M300US10140,"Aberdeen, WA Micro Area",0.427671,0.520744,0.429274,1.377689
2,310M300US10180,"Abilene, TX Metro Area",0.481092,0.496874,0.454192,1.432157
3,310M300US10220,"Ada, OK Micro Area",0.466566,0.526206,0.425964,1.418737


For each **M**etropolitan **S**tatistical **A**rea (MSA), a sustainability index is calculated based on economic, social, and environmental indicators (also indices).

$$\text{sustainability index} = \text{economic} + \text{social} + \text{environmental}$$

The domain indices are computed from a large number of development indicator variables. To keep the application simple, we'll work with this derived data instead of the underlying variables.

_**If you're interested,**_ you can dig deeper on the [Sustainable Development Report website](https://www.sustainabledevelopment.report/), which provides detailed data reports related to the U.N.'s 2030 sustainable development goals.

## What is covariation?

Last week we talked about exploring variation among individual variables: the tendency of values to change between observations.

**Covariation** refers to _**the tendency of two variables to change together between observations**_. Covariation is about relationships.

In [49]:
# scatterplot of social vs economic indices
econ_social = alt.Chart(city_sust).mark_point().encode(
    x = alt.X('Econ_Domain', scale = alt.Scale(zero = False)),
    y = alt.Y('Social_Domain', scale = alt.Scale(zero = False))
).properties(
    width = 350,
    height = 200
)

In [50]:
econ_social

alt.Chart(...)

The social and economic indices do seem to *vary together*: higher values of the economic index coincide with higher values of the social index.

That's all there is to it. _**You've already thought about covariation, even if you didn't know it!**_

## How is covariation measured?

Let $(x_1, y_1) \dots, (x_n, y_n)$ denote $n$ values of two variables, $X$ and $Y$.

If $X$ and $Y$ tend to vary together, then whenever $X$ is far from its mean, so is $Y$.

* In other words, *their deviations from typical values coincide*.

This coincidence (or lack thereof) can be captured quantiatively by the (sample) **covariance**:

$$\text{cov}(\mathbf{x}, \mathbf{y}) = \frac{1}{n - 1}\sum_{i = 1}^n (x_i - \bar{x})(y_i - \bar{y})$$ 

In [51]:
xy = city_sust.iloc[:, 2:4] # x = econ, y = social
xy_ctr = xy - xy.mean() # center by subtracting column means
xy_ctr.head(3)

,Econ_Domain,Social_Domain
0,0.098892,0.029696
1,-0.038701,-0.040819
2,0.014720,-0.064690


In [52]:
xy_cov = xy_ctr.prod(axis = 1).sum()/(len(xy) - 1) # cov(x, y)
xy_cov

0.0019850231634313846

## Correlation: standardized covariance

Covariance is a little tricky to interpret. Is 0.00199 large or small?

It is a little more useful to compute the (sample) **correlation**:

$$\text{corr}(\mathbf{x}, \mathbf{y}) = \frac{\text{cov}(\mathbf{x}, \mathbf{y})}{S_x S_y}$$

This is simply _**covariance standardized to [-1, 1]**_.

Properties:

* $\text{corr}(\mathbf{x}, \mathbf{x}) = 1$, since any variable's deviations coincide *perfectly* with themselves

* the sign indicates whether $X$ and $Y$ vary together or in opposition

* $\text{corr}(\mathbf{x}, \mathbf{y}) = 1, -1$ are the *strongest possible* correlations 

* $\text{corr}(\mathbf{x}, \mathbf{y}) = 0$ is the weakest possible correlation

* moderate to large correlations indicate that $X$ and $Y$ covary

* small correlations *do not indicate* that $X$ and $Y$ don't covary

## Correlation of social and economic indices

Standardizing the covariance makes it more interpretable:

In [53]:
xy_cov/xy.std().prod() # cov(x, y)/(sx*sy)

0.531490637165894

The correlation indicates that the social and economic indices vary together (positive) weakly (moderate magnitude on [0, 1] scale). 

_**This is just a number that quantifies what you already knew from the graphic**_: there is a positive relationship.

In [54]:
econ_social

alt.Chart(...)

## What is the use of a quantitative measure of covariation?

It helps to have a number so that we can talk precisely about the *strength* of a relationship, and compare relationships quantitatvely.

Consider looking at all pairwise relationships for patterns of covariation:

In [55]:
# extract social and economic indices
x_mx = city_sust.iloc[:, 2:5]

# long form dataframe for plotting panel
scatter_df = x_mx.melt(
    var_name = 'row',
    value_name = 'row_index'
).join(
    pd.concat([x_mx, x_mx, x_mx], axis = 0).reset_index(),
).drop(
    columns = 'index'
).melt(
    id_vars = ['row', 'row_index'],
    var_name = 'col',
    value_name = 'col_index'
)

# panel
scatter_panel = alt.Chart(scatter_df).mark_point(opacity = 0.4).encode(
    x = alt.X('row_index', scale = alt.Scale(zero = False), title = ''),
    y = alt.Y('col_index', scale = alt.Scale(zero = False), title = '')
).properties(
    width = 150, 
    height = 75
).facet(
    column = alt.Column('col', title = ''),
    row = alt.Row('row', title = '')
).resolve_scale(x = 'independent', y = 'independent')

In [56]:
scatter_panel

alt.FacetChart(...)

Is the economic index *more* related to the environmental index or the social index?

## Correlation matrix (example)

The pairwise correlations among all three variables can be represented in a simple square matrix:

In [57]:
x_mx.corr() # correlation matrix in pandas

,Econ_Domain,Social_Domain,Env_Domain
Econ_Domain,1.000000,0.531491,0.011206
Social_Domain,0.531491,1.000000,-0.138674
Env_Domain,0.011206,-0.138674,1.000000


The strongest relationship is between social and economic indices; the weakest is between environmental and economic indices.

_**Notice**_ the correspondence between this matrix and the scatterplot panel -- there is one number per plot that describes the strenth of the relationship shown visually.

## Correlation matrix (definition)

The (sample) correlation matrix can be expressed as a simple matrix product; let $\mathbf{X} \in \mathbb{R}^{n\times p}$ denote the data values for $n$ observations of $p$ variables.

Consider the centered and scaled version of the data: $\mathbf{Z} = \left\{\frac{x_{ij} - \bar{x}_j}{s_{x_j}}\right\}$. 

The (sample) **correlation matrix** is defined as:

$$\text{corr}(\mathbf{X}) = \underbrace{\frac{1}{n - 1}\mathbf{Z}'\mathbf{Z}}_{\text{call this } \mathbf{R}}$$

In [58]:
# correlation matrix 'by hand'
n = len(x_mx) # sample size
z_mx = (x_mx - x_mx.mean())/x_mx.std() # (xi - xbar)/sx
z_mx.transpose().dot(z_mx)/(n - 1) # Z'Z/(n - 1)

,Econ_Domain,Social_Domain,Env_Domain
Econ_Domain,1.000000,0.531491,0.011206
Social_Domain,0.531491,1.000000,-0.138674
Env_Domain,0.011206,-0.138674,1.000000


## Heatmap visualization

Often it's useful to visualize the correlation matrix as a heatmap.

In [59]:
# store correlation matrix
corr_mx = x_mx.corr()

# melt to long form
corr_mx_long = corr_mx.reset_index().rename(
    columns = {'index': 'row'}
).melt(
    id_vars = 'row',
    var_name = 'col',
    value_name = 'Correlation'
)

# visualize
heatmap = alt.Chart(corr_mx_long).mark_rect().encode(
    x = alt.X('col', title = '', sort = {'field': 'Correlation', 'order': 'ascending'}),
    y = alt.Y('row', title = '', sort = {'field': 'Correlation', 'order': 'ascending'}),
    color = alt.Color('Correlation', 
                      scale = alt.Scale(scheme = 'blueorange',
                                        domain = (-1, 1), 
                                        type = 'sqrt'),
                     legend = alt.Legend(tickCount = 5))
).properties(width = 200, height = 200)

In [60]:
heatmap

alt.Chart(...)

* Each cell corresponds to a pair of variables

* Cells are colored acccording to the magnitude of correlation between the pair

* Rows and columns are sorted in order of correlation strength

_**Notice**_ again the correspondence with the scatterplot panel.

# Eigendecomposition
* The eigenvalue problem
* Geometric interpretation
* Computations

## Why are we talking about eigendecomposition?

We are going to factor the correlation matrix into components. This is, in essence, PCA.

This is useful for two reasons:

* identifying which variables drive variation and covariation

* reducing correlation structure among many variables to simpler components;

* visualizing multivariate data.

The eigendecomposition provides a means of factoring the matrix.

## The eigenvalue problem

Let $\mathbf{A}$ be a square $(n\times n)$ matrix. 

The **eigenvalue problem** refers to finding nonzero $\lambda$ and $\mathbf{x}$ that satisfy the equation:

$$\mathbf{Ax} = \lambda\mathbf{x}$$

For any such solutions:
* $\lambda$ is an **eigenvalue** of $\mathbf{A}$;
* $\mathbf{x}$ is an **eigenvector** of $\mathbf{A}$.

## Geometry

For a simple example, suppose $n = 2$ and $\mathbf{x}$ is an eigenvalue of $\mathbf{A}$. Then:

$$
\mathbf{Ax} 
= \left[\begin{array}{cc}
        a_{11} & a_{12} \\
        a_{21} & a_{22}
      \end{array}\right]
      \left[\begin{array}{c} 
        x_1 \\ x_2 
      \end{array}\right]
= \left[\begin{array}{c} 
        a_{11} x_1 + a_{12} x_2 \\
        a_{21} x_1 + a_{22} x_2 
      \end{array}\right]
= \left[\begin{array}{c}
        \lambda x_1 \\ \lambda x_2 
        \end{array}\right]
= \lambda\mathbf{x}
$$

So the eigenvalue problem equation says that the *linear transformation of $\mathbf{x}$ by $\mathbf{A}$* is simply a rescaling of $\mathbf{x}$ by a factor of $\lambda$.

<img src='figures/eigenvalue-wiki.png' style='width:300px'>

## The decomposition

If $\lambda, \mathbf{x}$ are an eigenvalue and eigenvector of $\mathbf{A}$, then so are $c\lambda, c\mathbf{x}$ for any constant $c$; so usually attention is constrained to solutions such that $\|\mathbf{x}\| = 1$; these are unique to within the sign of $\mathbf{x}$.

The **eigendecomposition** of a square matrix consists in _**finding its unique nonzero eigenvalues and (unit-length) eigenvectors**_. 

This task is considered a 'decomposition' because the eigenvectors $\mathbf{V}$ and eigenvalues $\Lambda$ satisfy

$$\underbrace{\mathbf{A}}_{\text{original matrix}} = \underbrace{\mathbf{V}\Lambda\mathbf{V}'}_{\text{eigendecomposition}}$$

So the original matrix can be *reconstructed* from the eigenvalues and eigenvectors.

## Special case

We will be applying eigendecomposition to correlation matrices. These are of the form $\mathbf{A} = \mathbf{Z'Z}$. 

Such matrices have some special properties, with the consequences:
* $\mathbf{V'V} = \mathbf{I}$, in other words, the eigenvectors are an orthonormal basis;
* $\lambda_i \geq 0$, in other words, all eigenvalues are nonnegative.

## Computations

The eigendecomposition is computed numerically using iterative methods. Luckily, these are very easy to implement:

In [61]:
# compute decomposition
decomp = linalg.eig(corr_mx)
decomp

(array([0.44788559+0.j, 1.54664121+0.j, 1.0054732 +0.j]),
 array([[ 0.68276586, -0.68571102,  0.2522522 ],
        [-0.70523279, -0.7087523 , -0.01780118],
        [-0.19099079,  0.16574249,  0.96749778]]))

In [62]:
# eigenvalues
decomp[0]

array([0.44788559+0.j, 1.54664121+0.j, 1.0054732 +0.j])

In [63]:
# eigenvectors
decomp[1]

array([[ 0.68276586, -0.68571102,  0.2522522 ],
       [-0.70523279, -0.7087523 , -0.01780118],
       [-0.19099079,  0.16574249,  0.96749778]])

## Does the decomposition really work?

Let's check that in fact $\mathbf{A} = \mathbf{V\Lambda V'}$:

In [64]:
# store eigenvalues and eigenvectors as dataframes
eigenvecs = pd.DataFrame(decomp[1], index = corr_mx.columns, columns = ['v1', 'v2', 'v3'])
eigenvals = pd.DataFrame(np.diag(decomp[0].real), index = ['v1', 'v2', 'v3'], columns = ['v1', 'v2', 'v3'])

In [65]:
eigenvecs.dot(eigenvals).dot(eigenvecs.transpose()) # V Lambda V'

,Econ_Domain,Social_Domain,Env_Domain
Econ_Domain,1.000000,0.531491,0.011206
Social_Domain,0.531491,1.000000,-0.138674
Env_Domain,0.011206,-0.138674,1.000000


In [66]:
corr_mx # correlation matrix (our 'A')

,Econ_Domain,Social_Domain,Env_Domain
Econ_Domain,1.000000,0.531491,0.011206
Social_Domain,0.531491,1.000000,-0.138674
Env_Domain,0.011206,-0.138674,1.000000


## Orthogonality

Let's check also that in fact $\mathbf{V'V} = \mathbf{I}$:

In [67]:
eigenvecs.transpose().dot(eigenvecs) # V'V

,v1,v2,v3
v1,1.000000e+00,-5.311862e-16,-7.332327e-17
v2,-5.311862e-16,1.000000e+00,6.231243e-17
v3,-7.332327e-17,6.231243e-17,1.000000e+00


The significance of this property is that $\mathbf{v}_1, \mathbf{v}_2, \mathbf{v}_3$ form an *orthonormal basis*.

## What's a basis?

A **basis** in linear algebra is a _**set of vectors that span a linear space**_. Think of a basis as a set of axes.

The usual basis for $\mathbb{R}^3$ are the unit vectors:
$$\mathbf{e}_1 = (1, 0, 0) \quad \mathbf{e}_2 = (0, 1, 0) \quad \mathbf{e}_3 = (0, 0, 1)$$

Coordinates in $\mathbb{R}^3$ are usually represented as multiples of these basis vectors:
$$(1, 2, 1) = 1\mathbf{e}_1 + 2\mathbf{e}_2 + 1\mathbf{e}_3$$

**Terminology**:

* A basis is *orthogonal* if *all basis vectors are at right angles to one another*.

* A basis is *orthonormal* if *it is orthogonal and basis vectors are of unit length*.

## Nonstandard bases

To an extent the usual choice of $\mathbf{e}_1, \mathbf{e}_2, \mathbf{e}_3$ is arbitrary.

It's possible to choose *any* system based on three directions to represent the same point relative to the origin.

<img src = 'figures/basis-wiki.png' style = 'width:150px'>

## Interpreting a change of basis

_**Different bases are simply different coordinate systems.**_

It's akin to the possibility of locating the distance to the corner store by different units and relative to different directions. For example:

* half a block to the right from the front door; or

* sixty steps diagonally in a straight line; or

* fifteen steps north, fifty steps east.

## Eigendecomposition of the correlation matrix

So, decomposing the correlation matrix yields a basis.

It turns out that when data are represented on that basis, patterns of covariation vanish.

> Let $\mathbf{Y} = \mathbf{XV}$; this is the representation of $\mathbf{X}$ on the basis given by $\mathbf{V}$. Then
$$\mathbf{Y'Y} = \mathbf{V'X'XV} =\mathbf{V'V\Lambda V' V} = \Lambda$$
which implies that $\text{cov}(\mathbf{y}_j, \mathbf{y}_k) = 0$ since $\Lambda$ is diagonal.

So the change of basis to $\mathbf{V}$ in a sense 'captures' the covariation in the data. 

Notice that the change of basis is a linear transformation $\mathbf{XV}$.

This is the essence of PCA: using eigendecomposition to obtain a linear transformation that captures covariation.

# Principal components analysis

* What is PCA exactly?
* PCA in the low-dimensional setting
* Variation capture and loss
* Interpreting principal components
* Example application

## Principal components analysis

**Principal components analysis** (PCA) consists in _**finding a linear transformation of one's data that captures covariation**_.

Applications are varied, and so are descriptions of just what the method is supposed to do, but the core technique is simple:

1. Compute the eigendecomposition of the correlation matrix.
2. Choose a subset of eigenvectors.
3. Then do other stuff (the 'analysis' part).

**PCA terminology**:

* Eigenvectors are called 'principal component directions'

* The values of eigenvectors are called 'loadings'

* The projected data consist of the 'principal components'

## PCA in the low-dimensional setting

Let's consider what happens if we follow the steps of PCA outlined above with just two variables, say the social index and the economic index, which were the most correlated pair in the example data. 

Denote the observations on these two variables by
$$\mathbf{X} = [\mathbf{x}_1 \; \mathbf{x}_2] \qquad\text{where}\qquad \mathbf{x}_1 = \text{social index} \;\text{and}\; \mathbf{x}_2 = \text{economic index}$$

To get the correlation matrix, first compute $\mathbf{Z} = \left\{\frac{x_i - \bar{x}}{s_x}\right\}$.

In [68]:
# scatterplot of unscaled data
raw = alt.Chart(x_mx).mark_point(opacity = 0.1, color = 'black').encode(
    x = alt.X('Social_Domain'),
    y = alt.Y('Econ_Domain')
).properties(width = 200, height = 200, title = 'original data (X)')

# mean vector
mean = alt.Chart(
    pd.DataFrame(x_mx.mean()).transpose()
).mark_circle(color = 'red', size = 100).encode(
    x = alt.X('Social_Domain'),
    y = alt.Y('Econ_Domain')
)

# scatterplot of centered and scaled data
centered = alt.Chart(z_mx).mark_point(opacity = 0.1, color = 'black').encode(
    x = alt.X('Social_Domain'),
    y = alt.Y('Econ_Domain')
).properties(width = 200, height = 200, title = 'centered and scaled (Z)')

# mean vector
mean_ctr = alt.Chart(
    pd.DataFrame(z_mx.mean()).transpose()
).mark_circle(color = 'red', size = 100).encode(
    x = alt.X('Social_Domain'),
    y = alt.Y('Econ_Domain')
)

# lines at zero
axbase = alt.Chart(
    pd.DataFrame({'Social_Domain': 0, 'Econ_Domain': 0}, index = [0])
)
ax1 = axbase.mark_rule().encode(x = 'Social_Domain')
ax2 = axbase.mark_rule().encode(y = 'Econ_Domain')

#layer
plot = (raw + mean + ax1 + ax2) 
plot

alt.LayerChart(...)

In [69]:
(centered + mean_ctr + ax1 + ax2)

alt.LayerChart(...)

In [70]:
plot.configure_axis(domain = False)

alt.LayerChart(...)

The red dots indicate the means.

## PCA in the low-dimensional setting

Now we'll compute the eigendecomposition. This is pretty much all there is to it.

In [71]:
z_mx = z_mx.iloc[:, 0:2]

In [72]:
# compute correlation mx
r_mx = z_mx.transpose().dot(z_mx)/(len(z_mx) - 1)
r_mx

,Econ_Domain,Social_Domain
Econ_Domain,1.000000,0.531491
Social_Domain,0.531491,1.000000


In [73]:
# decomposition
r_eig = linalg.eig(r_mx.values)

# show PC directions
directions = pd.DataFrame(r_eig[1], columns = ['PC1_Direction', 'PC2_Direction'], index = r_mx.columns)
directions

,PC1_Direction,PC2_Direction
Econ_Domain,0.707107,-0.707107
Social_Domain,0.707107,0.707107


Remember that the principal component directions are just the eigenvectors.

## Geometry of PCA

What did we just do? Let's plot the principal component directions on the centered and scaled data.

In [74]:
# store directions as dataframe for plotting
direction_df = pd.DataFrame(
    np.vstack([np.zeros(2), r_eig[1][:, 0], np.zeros(2), r_eig[1][:, 1]]),
    columns = ['Econ_Domain', 'Social_Domain']
).join(
    pd.Series(np.repeat(['PC1', 'PC2'], 2), name = 'PC direction')
)

# plot directions as vectors
eigenbasis = alt.Chart(direction_df).mark_line(order = False).encode(
    x = 'Social_Domain', 
    y = 'Econ_Domain', 
    color = alt.Color('PC direction', scale = alt.Scale(scheme = 'dark2'))
)

# combine with scatter
centered_plot = (centered.properties(width = 300, height = 300) + mean_ctr + ax1 + ax2)

In [75]:
centered_plot + eigenbasis

alt.LayerChart(...)

Do you notice any relationship between the directions and the scatter?

## Geometry of PCA

How about now?

In [76]:
# scale directions by eigenvalues
direction_df = pd.DataFrame(
    np.vstack([np.zeros(2), r_eig[1][:, 0]*r_eig[0][0].real, np.zeros(2), r_eig[1][:, 1]*r_eig[0][1].real]),
    columns = ['Econ_Domain', 'Social_Domain']
).join(
    pd.Series(np.repeat(['PC1', 'PC2'], 2), name = 'PC direction')
)

# repeat plot
eigenbasis = alt.Chart(direction_df).mark_line(order = False).encode(
    x = 'Social_Domain', 
    y = 'Econ_Domain', 
    color = alt.Color('PC direction', scale = alt.Scale(scheme = 'dark2'))
)

In [77]:
centered_plot + eigenbasis

alt.LayerChart(...)

The difference here comes from scaling the directions by the corresponding eigenvalues (plotting $\lambda_1\mathbf{v}_1$ and $\lambda_2\mathbf{v}_2$).

*The principal component directions are the axes along which the data vary most, and the eigenvalues give the magnitude of that variation.*

## Geometry of PCA: projection

So if we wanted to look at just *one* quantity that retains variation and covariation in the original data, we could:

* project the data onto the new basis 

* and look at the values on the axis of the first PC direction (the one with the most variation, *i.e.*, largest $\lambda_i$).

In [78]:
# project data onto pc directions
pcdata = z_mx.dot(r_eig[1]).rename(columns = {0: 'PC1', 1: 'PC2'})

# scatterplot
projection = alt.Chart(pcdata).mark_point(opacity = 0.1, color = 'black').encode(
    x = alt.X('PC1', title = '', axis = None),
    y = 'PC2'
).properties(
    width = 300, 
    height = 200,
    title = 'Projected data'
)

# layer with univariate tick plot of pc1 values
pc1 = alt.Chart(pcdata).mark_tick(color = '#1B9E77').encode(x = 'PC1').properties(width = 300)

In [80]:
projection 

alt.Chart(...)

In [81]:
 pc1

alt.Chart(...)

If we select just the first principal component to 'do other stuff' with, we lose the variation in PC2. Is this a problem?

## Quantifying variation capture/loss

To figure out how much variation/covariation is captured and lost, we need to know how much is available in the first place. 

The **total variation in $\mathbf{X}$** is given in terms of the correlation matrix by:

$$\text{total variation} = \text{det}\left(\mathbf{R}\right) = \sum_{i = 1}^p \lambda_i$$

Now let $\mathbf{y}_j = \mathbf{Zv}_j$ be the $j$th principal component. Its variance is:

$$\frac{1}{n - 1}\mathbf{y}_j'\mathbf{y} = \frac{1}{n - 1}\mathbf{v}_j'\mathbf{Z'Zv}_j = \mathbf{v}_j'\mathbf{V'\Lambda Vv}_j = \mathbf{e}_j'\mathbf{\Lambda e}_j = \lambda_j$$

## Quantifying variation capture/loss

So the total variation is the sum of the eigenvalues, and the variance of each PC is the corresponding eigenvalue.

We can therefore define the **proportion of total variation explained** by the $j$th principal component as:

$$\frac{\lambda_j}{\sum_{j = 1}^p \lambda_j}$$

This is sometimes also called the *variance ratio* for the $j$th principal component.

So in our example:

In [84]:
eigenvalues = r_eig[0].real # store eigenvalues as real array
eigenvalues[0]/eigenvalues.sum() # variance ratio

0.765745318582947

The first principal component captures 77% of total variation.

## Interpreting principal components

So we have obtained a single derived variable that captures 3/4 of all variation and covariation in both variables. But what is this?

The values of the first principal component (green ticks) are given by:
$$\text{PC1}_i = \mathbf{x}_i'\mathbf{v}_1 = 0.7071 \times\text{economic}_i + 0.7071 \times\text{social}_i$$

So the principal component is a linear combination of the underlying variables. 

Those coefficients are known as **loadings**: they determine the _**relative weights**_ by which the variables contribute to the principal component.

In this case, the PC1 loadings are equal; so this principal component is simply the average of the social and economic indices.

## Example application

Now that we've reviewed the basic technique, we can consider cases with a larger number of variables. 

This is really where PCA is most useful.

* It can help answer the question: which variables are driving variation in the data?

* It can help reduce data dimensions by finding combinations of variables that preserve variation.

* It can provide a means of visualizing high-dimensional data.

So, let's look at a different example: world development indicators.

In [85]:
wdi = pd.read_csv('data/wdi-data.csv').iloc[:, 2:].set_index('country')
wdi.head(3)

,inequality_adjusted_life,inequality_adjusted_education,maternal_mortality,parliament_pct_women,labor_participation_women,labor_participation_men,total_pop,urban_pct_pop,pop_under5,pop_15to64,...,total_unemployment,youth_unemployment,prison_5yr,trade,foreign_investment,net_migration,immigrant_pop,tourists_2018,internet_users_2018,mobile_users_2018
country,,,,,,,,,,,,,,,,,,,,,
Norway,0.931,0.908,2,40.8,60.4,67.2,5.4,82.6,0.055556,0.648148,...,3.3,9.3,80,72.1,0.4,5.3,16.1,5688,96.5,107.2
Ireland,0.926,0.892,5,24.3,56.0,68.4,4.9,63.4,0.061224,0.653061,...,4.9,13.1,82,239.2,-20.4,4.9,17.1,10926,84.5,103.2
Switzerland,0.947,0.883,5,38.6,62.9,73.8,8.6,73.8,0.046512,0.662791,...,4.6,7.4,77,119.4,-2.6,6.1,29.9,10362,89.7,129.6


## Computation via sklearn.decomposition.PCA

Luckily, `sklearn` contains an easy-to-use implementation that saves the trouble of going through all the calculations we did before 'by hand'.

In [86]:
from sklearn.decomposition import PCA

The implementation simply requires centering and scaling the data first.

In [87]:
# center and scale
wdi_ctr = (wdi - wdi.mean())/wdi.std()

# compute principal components
pca = PCA(31)
pca.fit(wdi_ctr)

PCA(n_components=31)

## Variance ratios

Selecting the number of principal components to use is somewhat subjective, but always based on the variance ratios. 

* how many PCs capture substantial variation individually before the variance ratio 'tails off'?

* how many PCs are needed to capture a certain proportion of the total variation?

This can be assessed by plotting the variance ratios and their cumulative sum:

In [89]:
# store proportion of variance explained as a dataframe
pcvars = pd.DataFrame({'Proportion of variance explained': pca.explained_variance_ratio_})

# add component number as a new column
pcvars['Component'] = np.arange(1, 32)

# add cumulative variance explained as a new column
pcvars['Cumulative variance explained'] = pcvars.iloc[:, 0].cumsum(axis = 0)

# encode component axis only as base layer
base = alt.Chart(pcvars).encode(
    x = 'Component')

# make a base layer for the proportion of variance explained
prop_var_base = base.encode(
    y = alt.Y('Proportion of variance explained',
              axis = alt.Axis(titleColor = '#57A44C'))
)

# make a base layer for the cumulative variance explained
cum_var_base = base.encode(
    y = alt.Y('Cumulative variance explained', axis = alt.Axis(titleColor = '#5276A7'))
)

# add points and lines to each base layer
line = alt.Chart(pd.DataFrame({'Component': [2.5]})).mark_rule(opacity = 0.3, color = 'red').encode(x = 'Component')
prop_var = prop_var_base.mark_line(stroke = '#57A44C') + prop_var_base.mark_point(color = '#57A44C') + line
cum_var = cum_var_base.mark_line() + cum_var_base.mark_point() + line

# layer the layers
variance_plot = alt.layer(prop_var, cum_var).resolve_scale(y = 'independent') 

In [90]:
variance_plot.properties(height = 200, width = 400)

alt.LayerChart(...)

In this case, the variance ratios drop off after 2 components. These first two capture about half of total variation in the data.

## Loading plots

Examining the loadings can help address the question: *which variables drive variation in the data?*

In [91]:
# store the loadings as a data frame with appropriate names
loading_df = pd.DataFrame(pca.components_).transpose().rename(
    columns = {0: 'PC1', 1: 'PC2'}
).loc[:, ['PC1', 'PC2']]

# add a column with the taxon names
loading_df['indicator'] = wdi_ctr.columns.values

# melt from wide to long
loading_plot_df = loading_df.melt(
    id_vars = 'indicator',
    var_name = 'PC',
    value_name = 'Loading'
)

# create base layer with encoding
base = alt.Chart(loading_plot_df).encode(
    y = alt.X('indicator', title = ''),
    x = 'Loading',
    color = 'PC'
)

# store horizontal line at zero
rule = alt.Chart(pd.DataFrame({'Loading': 0}, index = [0])).mark_rule().encode(x = 'Loading', size = alt.value(2))

# layer points + lines + rule to construct loading plot
loading_plot = base.mark_point() + base.mark_line() + rule

In [92]:
loading_plot.properties(width = 300, height = 400)

alt.LayerChart(...)

## Visualization

Finally, we might want to use the first two principal components to visualize the data. Often it's helpful to merge the principal components with the original data and apply visualization techniques you already know to search for interesting patterns.

In [41]:
# project pcdata onto first two components; store as data frame
projected_data = pd.DataFrame(pca.fit_transform(wdi_ctr)).iloc[:, 0:2].rename(columns = {0: 'PC1', 1: 'PC2'})

# add index and reset
projected_data.index = wdi_ctr.index
projected_data = projected_data.reset_index()

# append one of the original variables
projected_data['gdppc'] = wdi.gdp_percapita.values
projected_data['pop'] = wdi.total_pop.values

# base chart
base = alt.Chart(projected_data)

# data scatter
scatter = base.mark_point().encode(
    x = alt.X('PC1:Q', title = 'Mortality'),
    y = alt.Y('PC2:Q', title = 'Labor'),
    color = alt.Color('gdppc', 
                      bin = alt.Bin(maxbins = 6), 
                      scale = alt.Scale(scheme = 'set2'), 
                      title = 'GDP per capita'),
    size = alt.Size('pop',
                   scale = alt.Scale(type = 'sqrt'),
                   title = 'Population')
).properties(width = 400, height = 400)

# text labels
label = projected_data.sort_values(by = 'gdppc', ascending = False).head(4)

text = alt.Chart(label).mark_text(align = 'left', dx = 3).encode(
     x = alt.X('PC1:Q', title = 'Mortality'),
    y = alt.Y('PC2:Q', title = 'Labor'),
    text = 'country'
)

In [42]:
scatter + text

alt.LayerChart(...)

# Summary

Our focus this week was on introducing PCA as an exploratory technique for understanding covariation in mulitvariate data.

By way of introduction, we discussed **covariation**, the tendency of values of multiple variables to change together.

* Measures of covariation: covariance and correlation

* Correlation matrices and heatmap visualization

We then moved on to **eigendecomposition**, a method of factoring matrices.

* For correlation (and covariance) matrices, eigendecomposition produces an orthonormal basis.

* Representing the data on this basis -- a linear transformation -- 'captures' covariation.

This provided the background needed to understand **principal components analysis** (PCA) -- finding covariation-preserving linear transformations of data.

* Technically amounts to eigendecomposition of a correlation matrix.

* Useful for identifying variables that drive covariation, and for visualizing multivariate data.